Записываем треки

In [86]:
import wave
import sys

import pyaudio

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1 if sys.platform == 'darwin' else 2
RATE = 44100
RECORD_SECONDS = 5

def record():
    with wave.open('voice_record.wav', 'wb') as wf:
        p = pyaudio.PyAudio()
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)

        stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True)

        print('Recording...')
        for _ in range(0, RATE // CHUNK * RECORD_SECONDS):
            wf.writeframes(stream.read(CHUNK))
        print('Done')

        stream.close()
        p.terminate()

In [87]:
import whisper

def recognise():
    model = whisper.load_model("small")
    return model.transcribe("voice_record.wav")
result = recognise()
print(result["text"])

 Покажи видео, если и шрек танцуют!


In [88]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3

nltk.download('stopwords')
nltk.download('punkt')

def preprocess(text: str):
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    tokenisation = [word for word in tokens if not word in stopwords.words('russian')]
    
    # Лемматизация
    morph = pymorphy3.MorphAnalyzer()
    result = list()
    for word in tokenisation:
        p = morph.parse(word)[0]
        result.append(p.normal_form)

    return result
text = result['text']
query = preprocess(text)
print(query)

['показать', 'видео', ',', 'шрека', 'танцевать', '!']


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lolomarka/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lolomarka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [112]:
import subprocess

def generate_url(query):
    print('generate_url')
    url_query = '%20'.join(query)                     #разделяем их через %20
    url = 'https://yandex.ru/search/?text=' + url_query + '&lr=213'   #подставляем
    print(url)

def search_youtube(query):
    print('search_youtube')
    url_query = '+'.join(query)
    url = 'https://www.youtube.com/results?search_query=' + url_query
    print(url)

def open_discord(_):
    print('open_discord')
    subprocess.call('Discord')

In [113]:
commands = {
    ("рассказать", "найти", ): generate_url,
    ("показать", "видео") : search_youtube,
    ('открывать', 'открыть') : open_discord
}

def proccess_command(command_name : str, text : str):
    print('Query: ', text)
    for key in commands.keys():
        if command_name in key:
            commands[key](text)
        else:
            print("Command not found")

In [114]:
def proccess():
    record()
    result = recognise()
    query = preprocess(result['text'])
    command = query[0]
    text = query[1:]
    print(command)
    print(text)
    proccess_command(command, text)

In [100]:
proccess()

Recording...
Done
рассказать
['размножаться', 'капибара', 'дикий', 'природа']
Query:  ['размножаться', 'капибара', 'дикий', 'природа']
generate_url
https://yandex.ru/search/?text=размножаться%20капибара%20дикий%20природа&lr=213
Command not found
Command not found


In [103]:
proccess()

Recording...
Done
показать
['видео', ',', 'youtube', 'ширек', 'танцевать', '!']
Query:  ['видео', ',', 'youtube', 'ширек', 'танцевать', '!']
Command not found
search_youtube
https://www.youtube.com/results?search_query=видео+,+youtube+ширек+танцевать+!
Command not found


In [115]:
proccess()

Recording...
Done
открыть
['дискорд', '!']
Query:  ['дискорд', '!']
Command not found
Command not found
open_discord


KeyboardInterrupt: 